# HW #3 Financial Ratio Quantile Strategies
[FINM 33150] Regression Analysis and Quantitative Trading Strategies\
Winter 2022 | Professor Brian Boonstra

_**Due:** Thursday, February 3rd, at 11:00pm\
**Name:** Ashley Tsoi (atsoi, Student ID: 12286230)_

### 1. Fetch and clean data

#### 1-1. Import packages

In [95]:
import os
import functools
import warnings

import quandl
import json
import pandas as pd
pd.set_option("display.precision", 4)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import math
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta

# let plot display in the notebook instead of in a different window
%matplotlib inline 
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [21, 8]

#### 1-2. Define the functions to fetch data from Quandl

**1-2-1. Get my personal keys** from ../data/APIs.json

In [2]:
f = open('../data/APIs.json')
APIs = json.load(f)
f.close()

**1-2-2. Define helper functions**

In [127]:
def assertCorrectDateFormat(date_text):
    try:
        dt.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect date format, should be YYYY-MM-DD")

def calcSixMonthsAgo(date_text):
    assertCorrectDateFormat(date_text)
    d = dt.datetime.strptime(date_text, '%Y-%m-%d')
    return (d + relativedelta(months=-6)).strftime('%Y-%m-%d')

def deleteCSV(sec):
    file_name = "../data_large/EOD/"+sec
    if os.path.isfile(file_name):
        os.remove(file_name)

**1-2-3. Define function** to retrieve raw data from Quandl

**Documentation:**
```
Zacks Fundamentals Collection B (ZFB)
https://data.nasdaq.com/databases/ZFB/documentation
https://data.nasdaq.com/databases/ZFB/usage/quickstart/python
```

In [4]:
# Define function that retrieves ZFB data from Quandl
@functools.lru_cache(maxsize=16) # Cache the function output
def getQuandlZFBData(from_table,secs,start_date,end_date,columns):
    # Get data fom Quandl using quandl.get_table
    # NOTE: missing data for the inputted date will NOT return a row.

    # INPUT         | DATA TYPE                 | DESCRIPTION
    # from_table    | string                    | FC, FR, MT, MKTV, SHRS, or HDM
    # secs          | string / tuple of string  | security ticker(s)
    # start_date    | string (YYYY-MM-DD)       | start date of data
    # end_date      | string (YYYY-MM-DD)       | end date of data (same as or after start_date)
    # columns       | string / tuple of string  | names of the columns to return
    
    if secs=='all': secs = list(pd.read_csv('../data/zacks-tickers.csv').ticker.unique()) # import all tickers from zacks-tickers

    if type(secs)==str: seclen = 1
    else: seclen=len(secs)
    print("Quandl | START | Retriving Quandl data for {:d} securities from the ZACKS/{} table: \n".format(seclen,from_table))
    
    # Retrieve data using quandl.get_table
    quandl.ApiConfig.api_key = APIs['Quandl']

    if from_table in ['FC','FR','MKTV','SHRS','HDM']:
        data = quandl.get_table('ZACKS/'+from_table,
                                ticker = secs, 
                                per_end_date = {'gte':start_date, 'lte':end_date},
                                qopts = {'columns':list(columns)},
                                paginate = True)
        
        data['per_end_date'] = pd.to_datetime(data['per_end_date'])
        if 'filing_date' in data.columns:
            data['filing_date'] = pd.to_datetime(data['filing_date'])

    elif from_table == 'MT':
        data = quandl.get_table('ZACKS/MT',
                                ticker = secs, 
                                qopts = {'columns':list(columns)},
                                paginate = True)

    else:
        print("from_table is limited to FC, FR, MT, MKTV, SHRS and HDM")
        
    print("Quandl | DONE  | Returning {:d} rows of data from the ZACKS/{} table.\n".format(len(data),from_table))

    return data


@functools.lru_cache(maxsize=16) # Cache the function output
def _getZFBData(secs,start_date,end_date):
    # Merged Zacks data in five tables: FC, FR, MT, MKTV, and SHRS
    # NOTE: missing data for the inputted date will NOT return a row.

    # INPUT         | DATA TYPE                 | DESCRIPTION
    # secs          | string / tuple of string  | security ticker(s)
    # start_date    | string (YYYY-MM-DD)       | start date of data
    # end_date      | string (YYYY-MM-DD)       | end date of data (same as or after start_date)
    
    # Retrieve data using quandl.get_table
    fc = getQuandlZFBData('FC',secs,start_date,end_date,('ticker','exchange','per_end_date','per_type','zacks_sector_code','basic_net_eps','diluted_net_eps','tot_lterm_debt','net_lterm_debt','filing_date'))
    fr = getQuandlZFBData('FR',secs,start_date,end_date,('ticker','exchange','per_end_date','per_type','ret_invst','tot_debt_tot_equity'))
    mt = getQuandlZFBData('MT',secs,start_date,end_date,('ticker','ticker_type','asset_type'))
    mktv = getQuandlZFBData('MKTV',secs,start_date,end_date,('ticker','per_end_date','per_type','mkt_val'))
    shrs = getQuandlZFBData('SHRS',secs,start_date,end_date,('ticker','per_end_date','per_type','shares_out','avg_d_shares'))

    # Merge the tables
    print("MERGE  | START | \n")

    zacks_1 = fc.merge(fr, how='outer', on=['ticker','exchange','per_end_date','per_type'])
    zacks_2 = mktv.merge(shrs, how='outer', on=['ticker','per_end_date','per_type'])
    zacks_3 = zacks_1.merge(zacks_2, how='outer', on=['ticker','per_end_date','per_type'])
    zacks = zacks_3.merge(mt, how='outer', on='ticker')

    print("MERGE  | DONE  | Returning {:d} rows of ZACKS data.\n".format(len(zacks)))
    
    return zacks


**Documentation**
```
End of Day US Stock Prices (EOD)
https://data.nasdaq.com/databases/EOD/documentation
https://data.nasdaq.com/databases/EOD/usage/quickstart/python
```

In [77]:
# Define function that retrieves EOD data from Quandl
@functools.lru_cache(maxsize=16) # Cache the function output
def getQuandlEODData(sec,start_date,end_date,columns):
    # Get one security (sec)'s data fom Quandl using quandl.get_table
    # NOTE: missing data for the inputted date will NOT return a row.

    # INPUT         | DATA TYPE                 | DESCRIPTION
    # sec           | string / list of string   | security ticker
    # start_date    | string (YYYY-MM-DD)       | start date of data
    # end_date      | string (YYYY-MM-DD)       | end date of data (same as or after start_date)
    # columns       | string / list of string   | columns to return
    
    print("Quandl | START | Retriving Quandl data for security: \n",sec)
    
    # Retrieve data using quandl.get_table
    quandl.ApiConfig.api_key = APIs['Quandl']
    data = quandl.get_table('QUOTEMEDIA/PRICES',
                            ticker = sec, 
                            date = {'gte':start_date, 'lte':end_date},
                            qopts = {'columns':list(columns)}
                            )

    print(f"Quandl | DONE  | Returning {len(data):d} dates of data for {sec}.\n")
    return data



def getAdjClose(secs,start_date,end_date):

    if type(secs)==str: secs = (secs,)

    data = []
    for sec in secs:
        file_name = "../data_large/EOD/"+sec
        if not os.path.isfile(file_name):
            # download as CSV in local directory
            getQuandlEODData(sec,start_date,end_date,('ticker','date','adj_close')).set_index(['date']).to_csv(file_name)
        
        data.append(pd.read_csv(file_name))
    
    data = pd.concat(data)
    data['date'] = pd.to_datetime(data['date'])
    
    return data.sort_values('date', ascending=True, ignore_index=True).set_index(['ticker','date'])



**1-2-4. Define function** to filter / clean raw data

**Requirements:**
```
- US Equities
- not in the automotive, financial or insurance sector over the entire period
- end-of-day adjusted closing prices are available over the entire period
- debt/market cap ratio is greater than 0.1
- has feasible calculation of the ratios over the entire period: 
  - debt to market cap, 
  - return on investment, and 
  - price to earnings. 
  Including for at least one PER END DATE no more than one year old. Debt ratio of zero is OK.
```

In [125]:
@functools.lru_cache(maxsize=16) # Cache the function output
def getCleanZFBData(secs,start_date,end_date):

    # === GET RAW DATA ============================================

    raw_zacks = _getZFBData(secs,start_date,end_date).rename(columns={'per_end_date':'date'})
    
    # === FILTER / CLEAN ==========================================
    
    # US Equities only
    zacks = raw_zacks[raw_zacks.exchange.isin(('NYSE','NASDAQ'))]  # select US stock exchanges
    zacks = zacks[zacks.ticker_type=='S']                          # S = Securities
    zacks = zacks[zacks.asset_type=='COM'][zacks.columns]          # COM = Common stocks
    # zacks.drop(['exchange','ticker_type','asset_type'], axis=1, inplace=True) # drop these columns as they are no longer needed

    # remove tickers without filing dates (tickers without filing dates are impossible to join on)
    filingDate_filter = zacks[pd.isnull(zacks.filing_date)].ticker.unique()
    zacks = zacks[~zacks.ticker.isin(filingDate_filter)]
    
    # not in the automotive, financial or insurance sector for any date (NOTE: there might be sector changes)
    sector_filter = zacks[zacks.zacks_sector_code.isin((5,13))].ticker.unique() # 5 = automotive, 13 = finance (includes insurance)
    zacks = zacks[~zacks.ticker.isin(sector_filter)]
    # zacks.drop(['zacks_sector_code'], axis=1, inplace=True) # drop these columns as they are no longer needed
    
    # Debt -- Use net debt where available, total debt otherwise
    zacks['debt'] = zacks.net_lterm_debt.fillna(zacks.tot_lterm_debt)

    # EPS -- use the basic version (GAAP) if no diluted number is available.
    zacks['eps'] = zacks.diluted_net_eps.fillna(zacks.basic_net_eps)
    zacks.eps.clip(lower=0.001, inplace=True) # make all negative eps 0.001

    # If have both quarterly & annual data for the same ticker & date, use quarterly
    ratio_cols = ['tot_debt_tot_equity','mkt_val','ret_invst','debt','eps','shares_out','avg_d_shares']
    zacks.set_index(['ticker','date'],inplace=True)
    zacks_quarterly = zacks[zacks.per_type=='Q']
    zacks_annual = zacks[zacks.per_type=='A']
    for c in ratio_cols:
        zacks_quarterly[c] = zacks_quarterly[c].fillna(zacks_annual[c])
    zacks.drop(['per_type'], axis=1, inplace=True) # drop these columns as they are no longer needed
    zacks = zacks_quarterly.reset_index()
    
    # debt-to-market-cap ratio greater than 0.1 AND not null (filter all since we will have enough tickers)
    badDebtToMC_filter = zacks[(zacks.tot_debt_tot_equity<=0.1) | (pd.isnull(zacks.tot_debt_tot_equity))].ticker.unique()
    zacks = zacks[~zacks.ticker.isin(badDebtToMC_filter)]

    # other ratios are not null (ret_invst, mkt_val, eps, debt)
    nullRatio_filter = list(zacks[pd.isnull(zacks.ret_invst) | pd.isnull(zacks.mkt_val)].ticker.unique())
    nullRatio_filter += list(zacks[pd.isnull(zacks.eps) | pd.isnull(zacks.debt)].ticker.unique())
    zacks = zacks[~zacks.ticker.isin(set(nullRatio_filter))]
    
    return zacks[['ticker','date']+ratio_cols]

@functools.lru_cache(maxsize=16) # Cache the function output
def getEodFundamentalData(secs,start_date,end_date):
    
    before_start_date = calcSixMonthsAgo(start_date)

    zacks = getCleanZFBData(secs,before_start_date,end_date)
    tickers = zacks.ticker.unique()
    prices = getAdjClose(tickers,before_start_date,end_date)
    prices.drop(['Unnamed: 0'], axis=1, inplace=True)
    
    # end-of-day adjusted closing prices are available
    noEOD_filter = set()
    
    for sec in tickers:
        price = prices.iloc[prices.index.get_level_values('ticker') == sec]
        data_len = len(price)
        if data_len < 1910 or any(pd.isnull(price.adj_close.loc[start_date:end_date])): # 1910 = number of trading days in the period 2013-07-01 -- 2021-01-31
            # filter out the security
            noEOD_filter.add(sec)
            if data_len > 0: pd.DataFrame().to_csv('../data_large/EOD/'+sec) # make csv of filtered securities an empty table so we skip downloading next time
    
    prices = prices.iloc[~prices.index.get_level_values('ticker').isin(noEOD_filter)]
    zacks = zacks[~zacks.ticker.isin(noEOD_filter)].set_index(['ticker','date'])
    
    # === JOIN PRICING DATA with FUNDAMENTAL DATA =================
    
    data = pd.concat([prices, zacks], axis=1)
    
    # === FORWARD FILL ============================================

    # forward fill equity price from the previous trading day if per_end_date is not a trading date
    data = data.transform(lambda v: v.ffill())
    
    index_ticker = data.index.get_level_values('ticker').unique()
    print(f'Returning EOD + fundamental data for {len(index_ticker)} tickers.')

    index_date = data.index.get_level_values('date')
    return data.loc[(index_date>=start_date) & (index_date<=end_date)]


In [128]:
test = getEodFundamentalData('all','2014-01-01','2021-01-31')
test

Quandl | START | Retriving Quandl data for 8913 securities from the ZACKS/FC table: 

Quandl | DONE  | Returning 203780 rows of data from the ZACKS/FC table.

Quandl | START | Retriving Quandl data for 8913 securities from the ZACKS/FR table: 

Quandl | DONE  | Returning 203780 rows of data from the ZACKS/FR table.

Quandl | START | Retriving Quandl data for 8913 securities from the ZACKS/MT table: 

Quandl | DONE  | Returning 7961 rows of data from the ZACKS/MT table.

Quandl | START | Retriving Quandl data for 8913 securities from the ZACKS/MKTV table: 

Quandl | DONE  | Returning 150762 rows of data from the ZACKS/MKTV table.

Quandl | START | Retriving Quandl data for 8913 securities from the ZACKS/SHRS table: 

Quandl | DONE  | Returning 168519 rows of data from the ZACKS/SHRS table.

MERGE  | START | 

MERGE  | DONE  | Returning 244830 rows of ZACKS data.

Returning EOD + fundamental data for 540 tickers.


adj_close  tot_debt_tot_equity    mkt_val  ret_invst  \
ticker date                                                               
AAP    2013-07-01    79.0805               0.4419  5922.0800     5.9232   
       2013-07-02    78.6749               0.4419  5922.0800     5.9232   
       2013-07-03    79.1770               0.4419  5922.0800     5.9232   
       2013-07-05    79.7468               0.4419  5922.0800     5.9232   
       2013-07-08    80.8477               0.4419  5922.0800     5.9232   
...                      ...                  ...        ...        ...   
ZTS    2021-01-25   160.4514               1.9080 78656.8200     3.4529   
       2021-01-26   158.8304               1.9080 78656.8200     3.4529   
       2021-01-27   152.3065               1.9080 78656.8200     3.4529   
       2021-01-28   155.6679               1.9080 78656.8200     3.4529   
       2021-01-29   153.4005               1.9080 78656.8200     3.4529   

                      debt    eps  shares_out  avg_d_shares  
ticker date                                                  
AAP    2013-07-01 604.1170 1.5900     72.9600       73.3400  
       2013-07-02 604.1170 1.5900     72.9600       73.3400  
       2013-07-03 604.1170 1.5900     72.9600       73.3400  
       2013-07-05 604.1170 1.5900     72.9600       73.3400  
       2013-07-08 604.1170 1.5900     72.9600       73.3400  
...                    ...    ...         ...           ...  
ZTS    2021-01-25 740.0000 0.7500    475.2700      478.7000  
       2021-01-26 740.0000 0.7500    475.2700      478.7000  
       2021-01-27 740.0000 0.7500    475.2700      478.7000  
       2021-01-28 740.0000 0.7500    475.2700      478.7000  
       2021-01-29 740.0000 0.7500    475.2700      478.7000  

[1035720 rows x 8 columns]

#### 1-3. Fetch cleaned data using the functions above

**Dates:**
```
January 1, 2014 - January 31, 2021*
```
**Note: fetch data from July 1, 2013 to get all data reported by January 1, 2014*

**1-3-1. Fetch data** 

In [ ]:
# zacks_fc = getQuandlZFBData('FC','all','2013-07-01','2021-01-31',('ticker','exchange','per_end_date','per_type','zacks_sector_code','basic_net_eps','diluted_net_eps','tot_lterm_debt','net_lterm_debt','filing_date'))
# zacks_fr = getQuandlZFBData('FR','all','2013-07-01','2021-01-31',('ticker','exchange','per_end_date','per_type','ret_invst','tot_debt_tot_equity'))
# zacks_mt = getQuandlZFBData('MT','all','2013-07-01','2021-01-31',('ticker','ticker_type','asset_type'))
# zacks_mktv = getQuandlZFBData('MKTV','all','2013-07-01','2021-01-31',('ticker','per_end_date','per_type','mkt_val'))
# zacks_shrs = getQuandlZFBData('SHRS','all','2013-07-01','2021-01-31',('ticker','per_end_date','per_type','shares_out','avg_d_shares'))

In [ ]:
l = [1,0,-1,-2]
[x if x>=0 else 0 for x in l ]